In [1]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.sentiment import vader
import numpy as np
import swifter

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
#nltk.download('vader_lexicon')
#nltk.download('stopwords')

In [3]:
df = pd.read_csv('../twitterscraper/tests/data/tweet/bitcoin/historic_scrape/raw/2015-02-01_2015-03-01.csv')

In [4]:
def cleanData(data):
    pattern = [ 'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+',  # URLs'
                '([^a-zA-Z0-9,.!? ]+?)', #anything else except text and punctuations
                ]

    sub_pattern = re.compile('|'.join(pattern))
    
    if isinstance(data, pd.Series):
        data = data.str.lower()
        replaced = data.str.replace(sub_pattern, '').str.strip()
    else:

        data = data.lower()
        replaced = re.sub(sub_pattern, '', word).strip()
        
    return replaced

In [5]:
df['Tweet'] = cleanData(df['Tweet'])

In [6]:
df['Time'] = pd.to_datetime(df['Time'], unit='s')

In [7]:
df = df.set_index('Time')

In [8]:
def applyVader(x, analyzer):
    return analyzer.polarity_scores(x)['compound']

In [9]:
analyzer = vader.SentimentIntensityAnalyzer()
df['sentiment'] = df['Tweet'].swifter.apply(applyVader, analyzer=analyzer)

Pandas Apply: 100%|██████████| 6066/6066 [00:00<00:00, 6132.37it/s]


In [43]:
def f(x):
    y = pd.Series()
    
    try:
        t = x.index[0]
        y['Time'] = pd.Timestamp("{}-{}-{} {}:00:00".format(t.year, t.month, t.day, t.hour))
    except:
        pass
    
    x = x.assign(f = x['Likes'] + x['Replies'] + x['Retweets']).sort_values('f', ascending=False).drop('f', axis=1)
    
    Nbullish = sum(x['sentiment'] > 0) #should use something else later
    Nbearish = sum(x['sentiment'] < 0)
    
    y['mean_vader_all'] = x['sentiment'].mean()
    
    #doi.org/10.1016/j.eswa.2016.12.036
    y['n_bullish_all'] = Nbullish
    y['n_bearish_all'] = Nbearish
    
    try:
        y['bullish_ratio_all'] = Nbullish / (Nbullish + Nbearish)
    except:
        y['bullish_ratio_all'] = np.nan
    
    try:
        y['bearish_ratio_all'] = Nbearish / (Nbullish + Nbearish)
    except:
        y['bearish_ratio_all'] = np.nan
        
    y['bullish_index_all'] = np.log((Nbullish + 1)/(Nbearish + 1))
    
    try:
        y['agreement_all'] = 1 - np.sqrt(1-(((Nbullish - Nbearish)/(Nbullish + Nbearish))**2) )
    except:
        y['agreement_all'] = np.nan
        
    try:
        y['spread_all'] = (Nbullish - Nbearish)/(Nbullish + Nbearish)
    except:
        y['spread_all'] = np.nan
    
    xTopAll = x.iloc[:int(x.shape[0] * .15)]
    
    y['mean_vader_top'] = xTopAll['sentiment'].mean()
    y['mean_likes_top'] = xTopAll['Likes'].mean()
    y['mean_replies_top'] = xTopAll['Replies'].mean()
    y['mean_retweets_top'] = xTopAll['Retweets'].mean()
    
    return y

In [44]:
from multiprocessing import Pool, cpu_count

In [45]:
def applyParallel(dfGrouped, func):
    retLst = Parallel(n_jobs=multiprocessing.cpu_count())(delayed(func)(group) for name, group in dfGrouped)
    return pd.DataFrame(retLst)

In [46]:
%time aa = applyParallel(df.groupby(pd.Grouper(freq='H')), f)

CPU times: user 491 ms, sys: 7.68 ms, total: 499 ms
Wall time: 2.59 s


In [47]:
aa

,Time,mean_vader_all,n_bullish_all,n_bearish_all,bullish_ratio_all,bearish_ratio_all,bullish_index_all,agreement_all,spread_all,mean_vader_top,mean_likes_top,mean_replies_top,mean_retweets_top
0,2015-02-01 00:00:00,0.235589,16.0,3.0,0.842105,0.157895,1.446919,0.270715,0.684211,0.108640,47.20,2.20,48.40
1,2015-02-01 01:00:00,0.312570,8.0,1.0,0.888889,0.111111,1.504077,0.371461,0.777778,0.440400,183.00,6.00,189.00
2,2015-02-01 02:00:00,-0.122186,2.0,2.0,0.500000,0.500000,0.000000,0.000000,0.000000,-0.868900,391.00,3.00,437.00
3,NaT,NaN,0.0,0.0,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-02-01 04:00:00,-0.030860,0.0,2.0,0.000000,1.000000,-1.098612,1.000000,-1.000000,NaN,NaN,NaN,NaN
5,2015-02-01 05:00:00,-0.131387,1.0,2.0,0.333333,0.666667,-0.405465,0.057191,-0.333333,-0.827100,223.00,22.00,277.00
6,2015-02-01 06:00:00,0.170660,2.0,1.0,0.666667,0.333333,0.405465,0.057191,0.333333,NaN,NaN,NaN,NaN
7,2015-02-01 07:00:00,0.111950,1.0,1.0,0.500000,0.500000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN
8,2015-02-01 08:00:00,0.839500,1.0,0.0,1.000000,0.000000,0.693147,1.000000,1.000000,NaN,NaN,NaN,NaN
9,2015-02-01 09:00:00,0.000000,0.0,0.0,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
t.year

2015

In [40]:
t.month

2

In [41]:
t.day

1

In [37]:
t.hour

1

Timestamp('2015-02-01 01:00:00')

In [47]:
newDf['variation_all'] = newDf['n_bullish_all'].diff()
newDf = newDf.drop(['n_bullish_all', 'n_bearish_all'], axis=1)
newDf['mean_vader_change_top'] = newDf['mean_vader_top'].diff()
#add botorNot too
newDf = newDf.replace(np.nan, 0)

In [182]:
newDf

,mean_vader_all,n_bullish_all,n_bearish_all,bullish_ratio_all,bearish_ratio_all,bullish_index_all,agreement_all,spread_all,mean_vader_top,mean_likes_top,mean_replies_top,mean_retweets_top
Time,,,,,,,,,,,,
2015-02-01 00:00:00,0.235589,16.0,3.0,0.842105,0.157895,1.446919,0.270715,0.684211,0.108640,47.20,2.20,48.40
2015-02-01 01:00:00,0.312570,8.0,1.0,0.888889,0.111111,1.504077,0.371461,0.777778,0.440400,183.00,6.00,189.00
2015-02-01 02:00:00,-0.122186,2.0,2.0,0.500000,0.500000,0.000000,0.000000,0.000000,-0.868900,391.00,3.00,437.00
2015-02-01 03:00:00,NaN,0.0,0.0,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
2015-02-01 04:00:00,-0.030860,0.0,2.0,0.000000,1.000000,-1.098612,1.000000,-1.000000,NaN,NaN,NaN,NaN
2015-02-01 05:00:00,-0.131387,1.0,2.0,0.333333,0.666667,-0.405465,0.057191,-0.333333,-0.827100,223.00,22.00,277.00
2015-02-01 06:00:00,0.170660,2.0,1.0,0.666667,0.333333,0.405465,0.057191,0.333333,NaN,NaN,NaN,NaN
2015-02-01 07:00:00,0.111950,1.0,1.0,0.500000,0.500000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN
2015-02-01 08:00:00,0.839500,1.0,0.0,1.000000,0.000000,0.693147,1.000000,1.000000,NaN,NaN,NaN,NaN


In [ ]:
#from data_adder import trends_ta function and then apply ta to mean_vader_top and mean_vader_all
newDf = trends_ta(newDf, 'mean_vader_top')
newDf = trends_ta(newDf, 'mean_vader_all')